In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
filepath = '/content/drive/My Drive/Colab Notebooks/datasets/GoogleNews-vectors-negative300.bin.gz'
textfilepath = '/content/drive/My Drive/Colab Notebooks/datasets/mdb/test'

In [0]:
import glob
import os
from random import shuffle
import numpy as np

from nltk.tokenize import TreebankWordTokenizer
from gensim.models.keyedvectors import KeyedVectors

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, Dense, Dropout, Activation

In [11]:
word_vector = KeyedVectors.load_word2vec_format(filepath, binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
maxlen = 400 # data to show the net before backpropagate and update weights
batch_size = 32
embedding_dims = 300 # Length of the token vectors to be passed to the convnet
filters = 250 # N of filters to train
kernel_size = 3 # with of the filters
hidden_dims = 250 # numbers of neurons
epochs = 2 # numer of times that the train data will be passed to the network

In [0]:
def pre_process_data(filepath):
    positive_path = os.path.join(filepath, 'pos')
    negative_path = os.path.join(filepath, 'neg')
    pos_label = 1
    neg_label = 0
    dataset = []
    
    print('Running...')

    for index, filename in enumerate(glob.glob(os.path.join(positive_path, '*.txt'))):
        print('Primeiro: {}'.format(index))
        with open(filename, 'r') as f:
            dataset.append((pos_label, f.read()))

    for index, filename in enumerate(glob.glob(os.path.join(negative_path, '*.txt'))):
        print('Segundo: {}'.format(index))
        with open(filename, 'r') as f:
            dataset.append((neg_label, f.read()))

    shuffle(dataset)

    print('DONE')
    
    return dataset

In [0]:
dataset = pre_process_data('/content/drive/My Drive/Colab Notebooks/datasets/mdb/test')

Running...
Primeiro: 0
Primeiro: 1
Primeiro: 2
Primeiro: 3
Primeiro: 4
Primeiro: 5
Primeiro: 6
Primeiro: 7
Primeiro: 8
Primeiro: 9
Primeiro: 10
Primeiro: 11
Primeiro: 12
Primeiro: 13
Primeiro: 14
Primeiro: 15
Primeiro: 16
Primeiro: 17
Primeiro: 18
Primeiro: 19
Primeiro: 20
Primeiro: 21
Primeiro: 22
Primeiro: 23
Primeiro: 24
Primeiro: 25
Primeiro: 26
Primeiro: 27
Primeiro: 28
Primeiro: 29
Primeiro: 30
Primeiro: 31
Primeiro: 32
Primeiro: 33
Primeiro: 34
Primeiro: 35
Primeiro: 36
Primeiro: 37
Primeiro: 38
Primeiro: 39
Primeiro: 40
Primeiro: 41
Primeiro: 42
Primeiro: 43
Primeiro: 44
Primeiro: 45
Primeiro: 46
Primeiro: 47
Primeiro: 48
Primeiro: 49
Primeiro: 50
Primeiro: 51
Primeiro: 52
Primeiro: 53
Primeiro: 54
Primeiro: 55
Primeiro: 56
Primeiro: 57
Primeiro: 58
Primeiro: 59
Primeiro: 60
Primeiro: 61
Primeiro: 62
Primeiro: 63
Primeiro: 64
Primeiro: 65
Primeiro: 66
Primeiro: 67
Primeiro: 68
Primeiro: 69
Primeiro: 70
Primeiro: 71
Primeiro: 72
Primeiro: 73
Primeiro: 74
Primeiro: 75
Primeiro: 7

In [0]:
def tokenize_and_vectorize(dataset):
    tokenizer = TreebankWordTokenizer()
    vectorized_data = []
    expected = []

    for sample in dataset:
        tokens = tokenizer.tokenize(sample[1])
        sample_vecs = []

        for token in tokens:
            try:
                sample_vecs.append(word_vector[token])

            except KeyError:
                pass

        vectorized_data.append(sample_vecs)

    return vectorized_data

def collect_expected(dataset):
    # expected results
    expected = []

    for sample in dataset:
        expected.append(sample[0])

    return expected

In [0]:
vectorized_data = tokenize_and_vectorize(dataset)
expected = collect_expected(dataset)

In [0]:
def pad_truncate(data, maxlen):
    """
    The dimension of the datas to be shown to the convolution
    network must have the same dimension size.
    This function add ZERO pading to the input dimension less
    then the maxlen
    """
    new_data = []

    zero_vector = []
    for _ in range(len(data[0][0])):
        zero_vector.append(0.0)

    for sample in data:
        if len(sample) > maxlen:
            temp = sample[:maxlen]
        elif len(sample) < maxlen:
            temp = sample
            # Append the appropriate number 0 vectors to the list
            additional_elems = maxlen - len(sample)
            for _ in range(additional_elems):
                temp.append(zero_vector)
        else:
            temp = sample
        new_data.append(temp)
    return new_data

In [0]:
maxlen = 400 # data to show the net before backpropagate and update weights
batch_size = 32
embedding_dims = 300 # Length of the token vectors to be passed to the convnet
filters = 250 # N of filters to train
kernel_size = 3 # with of the filters
hidden_dims = 250 # numbers of neurons
epochs = 2 # numer of times that the train data will be passed to the network

In [0]:
split_point = int(len(vectorized_data) * .8)

x_train = vectorized_data[:split_point]
y_train = expected[:split_point]
x_test = vectorized_data[split_point:]
y_test = expected[split_point:]

In [0]:
x_test = pad_truncate(x_test, maxlen)
x_train = pad_truncate(x_train, maxlen)

In [0]:
x_train = np.reshape(x_train, (len(x_train), maxlen, embedding_dims))
y_train = np.array(y_train)
x_test = np.reshape(x_test, (len(x_test), maxlen, embedding_dims))
y_test = np.array(y_test)

In [0]:
print('build model...')
model = Sequential([
    Conv1D(
        filters,
        kernel_size,
        padding='valid',
        activation='relu',
        strides=1,
        input_shape=(maxlen, embedding_dims)
    ),
    GlobalMaxPooling1D(),
    Dense(hidden_dims),
    Dropout(0.2),
    Activation('relu'),
    Dense(1),
    Activation('sigmoid')
])

W0807 21:19:18.490823 139682724964224 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


build model...


In [0]:
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

W0807 21:19:18.710963 139682724964224 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
model.fit(x_train, y_train,
         batch_size=batch_size,
         epochs=epochs,
         validation_data=(x_test, y_test))

Train on 20008 samples, validate on 5002 samples
Epoch 1/2


In [0]:
model_structure = model.to_json()
with open("cnn_model.json", "w") as json_file:
  json_file.write(model_structure)

model.save_weights("cnn_weights.h5")

In [0]:
from tensorflow.keras.models import model_from_json

In [ ]:
with open("cnn_model.json", "r") as json_file:
  json_string = json_file.read()

model = model_from_json(json_string)

In [0]:
model.load_weights('cnn_weights.h5')

In [0]:
sample = "I hate this song."

vec_list = tokenize_and_vectorize([(1, sample)])

test_vec_list = pad_truncate(vec_list, maxlen)

test_vec = np.reshape(test_vec_list, (len(test_vec_list), maxlen, embedding_dims))

model.predict(test_vec)